In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
source('paths.sh')
devtools::load_all(cud4_d)
devtools::load_all(dirname(dirname(snpnet_helper)))
source(snpnet_helper)


ℹ Loading cud4

ℹ Loading snpnet



In [3]:
traits_w_metrics_f %>%
fread() %>%
rename_with(
    function(x){str_replace(x, '#', '')}, starts_with("#")
) %>%
mutate(
    trait_label=sprintf('%s (%s)', trait_name, trait),
) -> traits_w_metrics_df


In [4]:
eval_full_f %>%
fread() %>%
rename_with(
    function(x){str_replace(x, '#', '')}, starts_with("#")
) -> eval_full_df


In [27]:
eval_full_df %>%
filter(
    trait %in% (
        traits_w_metrics_df %>% 
        filter(
            WBtest_is_significant,
            family == "binomial"
        ) %>%
        pull(trait)
    )
) %>%
select(trait, trait_category_plot, trait_name, split, case_n, control_n) %>%
unique %>%
mutate(
    case_ratio = case_n / (case_n + control_n),
    case_ratio_sd = (case_ratio * (1 - case_ratio) / (case_n + control_n)) ** 0.5
) -> sig_bin_traits_df


In [29]:
sig_bin_traits_df %>%
filter(
    split %in% c("train_val", "test")
) %>%
arrange(-case_n / (case_n + control_n)) %>% head

trait,trait_category_plot,trait_name,split,case_n,control_n,case_ratio,case_ratio_sd
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
BIN_FC30006164,Lifestyle and environment,Types of physical activity in last 4 weeks (other exercises),test,34031,3964,0.8956705,0.0015682471
BIN_FC30006164,Lifestyle and environment,Types of physical activity in last 4 weeks (other exercises),train_val,134747,15874,0.8946096,0.0007911785
BIN_FC7001747,Lifestyle and environment,"Hair colour (natural, before greying) brown",train_val,211969,57232,0.7874005,0.0007885707
BIN_FC7001747,Lifestyle and environment,"Hair colour (natural, before greying) brown",test,52905,14381,0.7862705,0.0015803593
BIN100240,Others,Coffee consumed,train_val,91145,25785,0.7794835,0.0012124423
BIN100240,Others,Coffee consumed,test,22541,6503,0.7760983,0.0024460134


In [58]:
case_freq_plot <- function(sig_bin_traits_df, split_target){
    sig_bin_traits_df %>%
    filter(
        split %in% c("train_val", split_target)
    ) %>%
    mutate(
        split2 = case_when(
            split == "train_val" ~ "train_val",
            split == split_target ~ "target",
            TRUE ~ "others"
        ),
        case_ratio = case_n / (case_n + control_n),
        case_ratio_sd = (case_ratio * (1 - case_ratio) / (case_n + control_n)) ** 0.5
    ) %>%
    select(trait, split2, case_ratio, case_ratio_sd) %>%
    pivot_wider(
        names_from = split2,
        values_from = c("case_ratio", "case_ratio_sd")
    ) %>%
    ggplot(aes(x = case_ratio_train_val, y = case_ratio_target)) +
    geom_errorbarh(aes(
        xmin = case_ratio_train_val - 1.96 * case_ratio_sd_train_val,
        xmax = case_ratio_train_val + 1.96 * case_ratio_sd_train_val,
    )) +
    geom_errorbar(aes(
        ymin = case_ratio_target - 1.96 * case_ratio_sd_target,
        ymax = case_ratio_target + 1.96 * case_ratio_sd_target,
    )) +
    geom_point() +
    theme_bw() +
    labs(
        title = "Case frequency of binary traits with significant PRS",
        x = "Score development set (white British)"
    )
}


In [66]:
for(ext in c("png", "pdf")){ggsave(
    sprintf("case_freq.test.%s", ext),
    sig_bin_traits_df %>%
    case_freq_plot("test") +
    labs(
        y = "hold-out test set (white British)"
    ),
    width=6, height=6
)}


In [67]:
for(ext in c("png", "pdf")){ggsave(
    sprintf("case_freq.4pops.%s", ext),
    gridExtra::arrangeGrob(
        sig_bin_traits_df %>%
        case_freq_plot("non_british_white") +
        labs(
            y = "non-British white"
        ),
        sig_bin_traits_df %>%
        case_freq_plot("african") +
        labs(
            y = "African"
        ),
        sig_bin_traits_df %>%
        case_freq_plot("s_asian") +
        labs(
            y = "South Asian"
        ),
        sig_bin_traits_df %>%
        case_freq_plot("e_asian") +
        labs(
            y = "East Asian"
        ),
        # format
        ncol=2
    ),
    width=12, height=12
)}



Warning message:
“Removed 1 rows containing missing values (geom_errorbarh).”
Warning message:
“Removed 1 rows containing missing values (geom_point).”
Warning message:
“Removed 15 rows containing missing values (geom_errorbarh).”
Warning message:
“Removed 15 rows containing missing values (geom_point).”
Warning message:
“Removed 1 rows containing missing values (geom_errorbarh).”
Warning message:
“Removed 1 rows containing missing values (geom_point).”
Warning message:
“Removed 15 rows containing missing values (geom_errorbarh).”
Warning message:
“Removed 15 rows containing missing values (geom_point).”
